## Quant Research:

* Analysing the distribution of a stock returns
* Deriving a stock beta
* Determining correlations between two or more stocks
* Calculating rate of change
* Backtesting a strategy

While choosing portfolio, choose uncorelated stocks

Decide your strategy as per the momentum of the market.


### Backtest Moving Average Crossover on multiple Stocks

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data = pd.read_csv('/home/rvl224/Downloads/aapl_daily_data.csv', index_col = 0, parse_dates = True)

In [5]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-12-31,1.159,1.161,1.143,1.150,0.991,278588800
2005-01-03,1.157,1.163,1.118,1.130,0.973,691992000
2005-01-04,1.139,1.169,1.124,1.142,0.983,1096810400
2005-01-05,1.151,1.165,1.144,1.152,0.992,680433600
2005-01-06,1.155,1.159,1.131,1.153,0.993,705555200


In [6]:
# calculate buy and hold returns 
data['d_returns'] = data['Adj Close'].pct_change()

In [10]:
# strategy params

sma = 10
lma = 20

In [9]:
#calculate rules - moving aveages

data['sma'] = data['Adj Close'].rolling(sma).mean()
data['lma'] = data['Adj Close'].rolling(lma).mean()

In [12]:
#calculate trading signals
data['signals'] = np.where(data['sma'] >= data['lma'], 1, -1)

In [13]:
#drop NA 

data.dropna(inplace=True)

In [ ]:
data[]

In [ ]:
import yfinance as yf

# define stock list

stock_list = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG']

